In [2]:
!pip install -U transformers
!pip install -U torch
!pip install pytorch_lightning
!pip install -U datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

**Load datasets**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import f1_score

# Load dataset
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.txt", delimiter='\t', header=None, names=['id',    'Turn1',    'Turn2',    'Turn3',    'label'])
dev_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.txt", delimiter='\t', header=None, names=['id',  'Turn1',    'Turn2',    'Turn3',    'label'])
data = data.drop(0)
data = data.head(7000)               #take only the first 7k conversations out of 30k
dev_data = dev_data.drop(0)
dev_data = dev_data.head(1200)       #take only the first 1.2k conversations out of 3k

**small letter**

In [6]:
data['Turn1'] = data['Turn1'].str.lower()
data['Turn1'] = data['Turn1'].str.strip()
data['Turn2'] = data['Turn2'].str.lower()
data['Turn2'] = data['Turn2'].str.strip()
data['Turn3'] = data['Turn3'].str.lower()
data['Turn3'] = data['Turn3'].str.strip()

dev_data['Turn1'] = dev_data['Turn1'].str.lower()
dev_data['Turn1'] = dev_data['Turn1'].str.strip()
dev_data['Turn2'] = dev_data['Turn2'].str.lower()
dev_data['Turn2'] = dev_data['Turn2'].str.strip()
dev_data['Turn3'] = dev_data['Turn3'].str.lower()
dev_data['Turn3'] = dev_data['Turn3'].str.strip()

**Convert Emojis**

In [7]:
!pip install emoji
import emoji

def replace_emojis_with_meanings(text):
    if isinstance(text, str):
        return emoji.demojize(text)
    else:
        return np.nan

data["Turn1"] = data["Turn1"].apply(replace_emojis_with_meanings)
data["Turn2"] = data["Turn2"].apply(replace_emojis_with_meanings)
data["Turn3"] = data["Turn3"].apply(replace_emojis_with_meanings)


dev_data["Turn1"] = dev_data["Turn1"].apply(replace_emojis_with_meanings)
dev_data["Turn2"] = dev_data["Turn2"].apply(replace_emojis_with_meanings)
dev_data["Turn3"] = dev_data["Turn3"].apply(replace_emojis_with_meanings)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 6.7 MB/s eta 0:00:00


**Convert Short forms**

In [8]:
!pip install contractions
import pandas as pd
import contractions

# Function to replace short forms using contractions library
def replace_short_forms(text):
    if isinstance(text, str):
        return contractions.fix(text)
    else:
        return text  # Return the text as it is if it's not a string

# Apply the function to the 'text' column
data['Turn1'] = data['Turn1'].apply(replace_short_forms)
data['Turn2'] = data['Turn2'].apply(replace_short_forms)
data['Turn3'] = data['Turn3'].apply(replace_short_forms)

dev_data['Turn1'] = dev_data['Turn1'].apply(replace_short_forms)
dev_data['Turn2'] = dev_data['Turn2'].apply(replace_short_forms)
dev_data['Turn3'] = dev_data['Turn3'].apply(replace_short_forms)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.2 MB/s eta 0:00:00


In [9]:
data.shape

(7000, 5)

**Custom Tokenizer & Encodings**

In [10]:
# Define emotion classes
# Define emotion classes
label_dict = {'happy': np.int64(0), 'angry': np.int64(1), 'sad': np.int64(2), 'others': np.int64(3)}

# Map labels to numeric values
data['label'] = data['label'].map(label_dict)
dev_data['label'] = dev_data['label'].map(label_dict)

# Split dataset into training and validation sets
#train_df, val_df = train_test_split(data, test_size=0.1, random_state=42)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def custom_tokenize(row, tokenizer):
    # Combine the turns with special tokens
    text = f"[CLS] {row['Turn1']} [SEP] {row['Turn2']} [SEP] {row['Turn3']} [SEP]"
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=False,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return {
        'input_ids': encoding['input_ids'].flatten().tolist(),
        'attention_mask': encoding['attention_mask'].flatten().tolist()
    }

# Apply custom tokenization to the training and validation data
train_encodings = data.apply(lambda x: custom_tokenize(x, tokenizer ), axis=1)
dev_encodings = dev_data.apply(lambda x: custom_tokenize(x, tokenizer), axis=1)

# Convert to lists
train_input_ids = [encoding['input_ids'] for encoding in train_encodings]
train_attention_masks = [encoding['attention_mask'] for encoding in train_encodings]
dev_input_ids = [encoding['input_ids'] for encoding in dev_encodings]
dev_attention_masks = [encoding['attention_mask'] for encoding in dev_encodings]

# Create PyTorch datasets
class ConversationDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)
train_dataset = ConversationDataset(train_input_ids, train_attention_masks, data['label'].tolist())
dev_dataset = ConversationDataset(dev_input_ids, dev_attention_masks, dev_data['label'].tolist())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
print(f"train_dataset[0]: {train_dataset[3]}")
print(f"val_dataset[0]: {dev_dataset[3]}")

train_dataset[0]: {'input_ids': tensor([  101,  2017,  1054,  9951,   102,  1045,  2453,  2022,  9951,  2021,
         1045,  2572,  4129,  1996,  3606,  1012,   102,  2017,  2210, 19424,
        17219,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,    

**MODEL Initialization**

In [12]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=2,  # Smaller batch size due to potential memory constraints
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Load pre-trained BERT model with a classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Training**

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.778400,0.712053
2,1.011600,0.718088
3,0.683900,0.902950
4,0.001700,0.804928
5,0.269400,0.802583


TrainOutput(global_step=17500, training_loss=0.43792397765139385, metrics={'train_runtime': 3004.7631, 'train_samples_per_second': 11.648, 'train_steps_per_second': 5.824, 'total_flos': 2302263075840000.0, 'train_loss': 0.43792397765139385, 'epoch': 5.0})

**Evaluation(Micro Avg. F1 score)**

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Make predictions on the validation set
preds_output = trainer.predict(dev_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

# True labels
true_labels = dev_data['label'].tolist()

# Calculate accuracy
accuracy = accuracy_score(true_labels, preds)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1 scores for each class and average methods
precision_micro = precision_score(true_labels, preds, average='micro')
recall_micro = recall_score(true_labels, preds, average='micro')
f1_micro = f1_score(true_labels, preds, average='micro')

precision_macro = precision_score(true_labels, preds, average='macro')
recall_macro = recall_score(true_labels, preds, average='macro')
f1_macro = f1_score(true_labels, preds, average='macro')

precision_weighted = precision_score(true_labels, preds, average='weighted')
recall_weighted = recall_score(true_labels, preds, average='weighted')
f1_weighted = f1_score(true_labels, preds, average='weighted')

print(f'Micro average Precision: {precision_micro}')
print(f'Micro average Recall: {recall_micro}')
print(f'Micro average F1 score: {f1_micro}')

print(f'Macro average Precision: {precision_macro}')
print(f'Macro average Recall: {recall_macro}')
print(f'Macro average F1 score: {f1_macro}')

print(f'Weighted average Precision: {precision_weighted}')
print(f'Weighted average Recall: {recall_weighted}')
print(f'Weighted average F1 score: {f1_weighted}')

# Detailed classification report
report = classification_report(true_labels, preds, target_names=label_dict.keys())
print(f'Classification Report:\n{report}')

# Confusion matrix
conf_matrix = confusion_matrix(true_labels, preds)
print(f'Confusion Matrix:\n{conf_matrix}')



Accuracy: 0.885
Micro average Precision: 0.885
Micro average Recall: 0.885
Micro average F1 score: 0.885
Macro average Precision: 0.6531470571989515
Macro average Recall: 0.809139229613404
Macro average F1 score: 0.7141889347458613
Weighted average Precision: 0.9116698869997756
Weighted average Recall: 0.885
Weighted average F1 score: 0.893986358858293
Classification Report:
              precision    recall  f1-score   support

       happy       0.59      0.75      0.66        59
       angry       0.52      0.87      0.65        61
         sad       0.53      0.72      0.61        50
      others       0.97      0.90      0.94      1030

    accuracy                           0.89      1200
   macro avg       0.65      0.81      0.71      1200
weighted avg       0.91      0.89      0.89      1200

Confusion Matrix:
[[ 44   1   0  14]
 [  0  53   3   5]
 [  2   4  36   8]
 [ 29  43  29 929]]


***Micro avg. f1 Score = 0.88***


